In [1]:
import pandas as pd
import numpy as np
import re
import scipy.stats as stats
from sklearn import linear_model
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier

In [2]:
la = pd.read_csv('listings.csv')
# la = pd.read_csv('listings_review_location_imputed.csv')

la.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,109,https://www.airbnb.com/rooms/109,20180908131025,2018-09-08,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...",none,NaN,...,f,NaN,NaN,f,f,strict_14_with_grace_period,t,f,1,0.02
1,344,https://www.airbnb.com/rooms/344,20180908131025,2018-09-08,Family perfect;Pool;Near Studios!,This home is perfect for families; aspiring ch...,"Cheerful & comfortable; near studios, amusemen...",This home is perfect for families; aspiring ch...,none,Quiet-yet-close to all the fun in LA! Hollywoo...,...,f,NaN,NaN,t,f,flexible,f,f,1,0.11
2,2404,https://www.airbnb.com/rooms/2404,20180908131025,2018-09-08,deleted,NaN,deleted,deleted,none,NaN,...,f,NaN,"{""City of Los Angeles"","" CA""}",f,f,moderate,f,f,1,NaN
3,2708,https://www.airbnb.com/rooms/2708,20180908131025,2018-09-08,Gold Memory Foam Bed & Breakfast in West Holly...,"Complementary, Peet's coffee, latte-style coff...",This space is completely upgraded and updated....,"Complementary, Peet's coffee, latte-style coff...",none,We are minutes away from the Mentor Language I...,...,f,NaN,"{""City of Los Angeles"","" CA""}",t,f,strict_14_with_grace_period,f,f,1,0.23
4,2732,https://www.airbnb.com/rooms/2732,20180908131025,2018-09-08,Zen Life at the Beach,NaN,This is a three story townhouse with the follo...,This is a three story townhouse with the follo...,none,NaN,...,t,228269,"{""Santa Monica""}",f,f,strict_14_with_grace_period,f,f,2,0.19


In [3]:
#feature selection
keep = ['host_is_superhost', 'host_total_listings_count', 'host_identity_verified',
        'zipcode','property_type','bathrooms', 'bedrooms', 'beds','price','cancellation_policy',
        'availability_30','availability_60','availability_90','availability_365',
        'reviews_per_month','review_scores_location']
la_sub = la[keep]

In [4]:
#check the number of NaNs for each feature
la_sub.isnull().sum()

host_is_superhost               7
host_total_listings_count       7
host_identity_verified          7
zipcode                       530
property_type                   0
bathrooms                      38
bedrooms                       21
beds                           43
price                           0
cancellation_policy             0
availability_30                 0
availability_60                 0
availability_90                 0
availability_365                0
reviews_per_month            8913
review_scores_location       9639
dtype: int64

#### Step 1,  clean up NaN values

1) Impute `review_scores_location`

a) Extract relevant col's from original dataset

In [5]:
listDtaSubset = la[['latitude','longitude',"neighbourhood_cleansed", "review_scores_location"]]

In [6]:
# listDtaSubset.shape
# listDtaSubset.isna().sum(axis = 0)

b) One-hot encode neighbourhood variable

In [7]:
neighbourhood = pd.get_dummies(listDtaSubset.neighbourhood_cleansed)  # a matrix with number of rows equal to the number of rows in the original dataset and number cols = number unique neighborhood values. For each row, only one element is 1, everything else are 0's.
listDtaSubset = listDtaSubset.join(neighbourhood)
listDtaSubset.drop(columns = "neighbourhood_cleansed", inplace= True)

In [8]:
# listDtaSubset.shape

c) Use subset of data to train KNN (rows with all model variable and label non-missing)

In [9]:
# drop all observations with either x or y missing w.r.t. the model
# Note: I remember none of the latitude, longiture info is missing
data = listDtaSubset.dropna(axis = 0, subset = ['latitude','longitude',"review_scores_location"])
# data.shape

d) Use KNN model to predict all location review scores

In [10]:
# split valid data set into training and testing data sets
tmp_train, tmp_test = model_selection.train_test_split(data, test_size = 0.3)
tmp_train_X = tmp_train.drop(columns="review_scores_location")
tmp_train_Y = tmp_train["review_scores_location"]

tmp_test_X = tmp_test.drop(columns="review_scores_location")
tmp_test_Y = tmp_test["review_scores_location"]

In [11]:
# tmp_test_X.shape
# tmp_train_Y.shape

In [12]:
# Train KNN model
KNN = KNeighborsClassifier(n_neighbors = 11)  #  I tried k = 3-5. 11 is significantly better than 3, but similar to 12-15 or 10

KNN.fit(tmp_train_X, tmp_train_Y)
KNN_train_score = KNN.score(tmp_train_X,tmp_train_Y)
KNN_test_score = KNN.score(tmp_test_X, tmp_test_Y)

print("training accuracy: ", KNN_train_score, "\ntesting accuracy: ", KNN_test_score)

training accuracy:  0.777399446691 
testing accuracy:  0.747169811321


e) Fill missing location review score with predicted missing value

In [13]:
data2 = listDtaSubset.dropna(axis = 0, subset = ['latitude','longitude'])  # should not make any change since none of the latitude and longitude is missing in LA listing dataset
# data2.shape
data2 = data2.drop(columns="review_scores_location")

# predict the location score for ALL observations
review_scores_location_imputed = KNN.predict(data2)

In [14]:
# For observation with NA as review score, substitue in the predicted score
mask = la["review_scores_location"].isna()
la.loc[mask,"review_scores_location"] = review_scores_location_imputed[mask]
# listDta = listDta.is (value = pd.Series(review_scores_location_imputed), axis = 0)

2) Clean up other features

In [15]:
la_sub = la[keep]
summary = pd.DataFrame(la_sub.isnull().sum())
man = ['Drop rows with NaN(7)','/','/','Drop rows with NaN(530)','/','Drop rows with NaN(37)','Drop rows with NaN(19)','Drop rows with NaN(21)','/','/','/','/','/','/','?','/']
summary['manipulation']=man
summary

,0,manipulation
host_is_superhost,7,Drop rows with NaN(7)
host_total_listings_count,7,/
host_identity_verified,7,/
zipcode,530,Drop rows with NaN(530)
property_type,0,/
bathrooms,38,Drop rows with NaN(37)
bedrooms,21,Drop rows with NaN(19)
beds,43,Drop rows with NaN(21)
price,0,/
cancellation_policy,0,/


In [16]:
#host_is_superhost #7 missing features
la[la['host_is_superhost'].isnull()][['host_id','host_is_superhost']]
la_sub = la_sub[pd.notnull(la_sub['host_is_superhost'])]
#host with missing values just do not have values for all of the listings
#Drop NaN values, then host_total_listings_count and host_identity_verified are cleaned up as well

In [17]:
#zipcode #drop the NaN values
la_sub = la_sub[pd.notnull(la_sub['zipcode'])]

In [18]:
la_sub = la_sub[pd.notnull(la_sub['bathrooms'])]
la_sub = la_sub[pd.notnull(la_sub['bedrooms'])]
la_sub = la_sub[pd.notnull(la_sub['beds'])].reset_index(drop=True)

In [19]:
#fill missing value in `reviews_per_month` with median value
sub = la_sub["reviews_per_month"].median()
la_sub["reviews_per_month"] = la_sub["reviews_per_month"].fillna(value =sub)

In [20]:
la_sub.isnull().sum()

host_is_superhost            0
host_total_listings_count    0
host_identity_verified       0
zipcode                      0
property_type                0
bathrooms                    0
bedrooms                     0
beds                         0
price                        0
cancellation_policy          0
availability_30              0
availability_60              0
availability_90              0
availability_365             0
reviews_per_month            0
review_scores_location       0
dtype: int64

#### Step 2, Features Transformation

In [21]:
##host_is_superhost: 0-False, 1-True
la_sub['host_is_superhost']=[0 if la_sub['host_is_superhost'][i]=='f' else 1 for i in range(la_sub.shape[0])]

In [22]:
##host_identity_verified: 0-False, 1-True
la_sub['host_identity_verified']= [0 if la_sub['host_identity_verified'][i]=='f' else 1 for i in range(la_sub.shape[0])]

In [23]:
##zipcode
la_sub['len']=[len(la_sub['zipcode'][i]) for i in range(la_sub.shape[0])]
la_sub['len'].unique()

array([ 5, 10, 12,  4,  6,  7])

In [24]:
#Case 1: with 'CA', len=7
la_sub['zipcode']=[la_sub['zipcode'][i][2:] if la_sub['len'][i]==7 else la_sub['zipcode'][i] for i in range(la_sub.shape[0])]
#Case 2: with '-', len=10
la_sub['zipcode']=[la_sub['zipcode'][i][0:5] if re.search(r'-',la_sub['zipcode'][i]) else la_sub['zipcode'][i] for i in range(la_sub.shape[0])]
#Case 3: with 'Near' or '/n/n'
la_sub['zipcode']=[re.sub("[^0-9]", "", la_sub['zipcode'][i]) for i in range(la_sub.shape[0])]
#Case 4: len>6, take first 5 digits
la_sub['zipcode']=[la_sub['zipcode'][i][0:5] if len(la_sub['zipcode'][i])>5 else la_sub['zipcode'][i] for i in range(la_sub.shape[0])]
#Case 5: len<5 drop it
la_sub['len2']=[len(la_sub['zipcode'][i]) for i in range(la_sub.shape[0])]
la_sub = la_sub[la_sub['len2']==5].reset_index(drop=True)

#drop len & len2
la_sub = la_sub.drop(['len','len2'],axis=1)

In [25]:
##cancellation_policy
la_sub['cancellation_policy'].unique()

array(['strict_14_with_grace_period', 'flexible', 'moderate',
       'super_strict_30', 'strict', 'super_strict_60'], dtype=object)

In [26]:
##cancellation_policy - only keep three levels: flexible, moderate, strict, and make them binary encoded
la_sub['cancellation_policy'] = la_sub['cancellation_policy'].replace(['strict_14_with_grace_period','super_strict_30','strict','super_strict_60'],'strict')
binary_encoded = pd.get_dummies(la_sub.cancellation_policy)
newcols = binary_encoded.columns
la_sub[newcols] = binary_encoded

In [27]:
###remove $ from price
la_sub['price'] = (la_sub['price'].str[1:])
la_sub['price'] = la_sub['price'].str.replace(',', '')
la_sub['price'] = la_sub['price'].astype(float)

In [28]:
la_sub.head()

,host_is_superhost,host_total_listings_count,host_identity_verified,zipcode,property_type,bathrooms,bedrooms,beds,price,cancellation_policy,availability_30,availability_60,availability_90,availability_365,reviews_per_month,review_scores_location,flexible,moderate,strict
0,0,1.0,1,90230,Condominium,2.0,2.0,3.0,122.0,strict,0,0,0,146,0.02,10.0,0,0,1
1,0,1.0,1,91505,House,1.0,3.0,3.0,168.0,flexible,4,9,9,62,0.11,10.0,1,0,0
2,0,1.0,1,90046,Apartment,1.5,1.0,1.0,79.0,strict,3,3,3,272,0.23,10.0,0,0,1
3,0,2.0,0,90405,Apartment,1.0,1.0,1.0,140.0,strict,7,7,29,304,0.19,10.0,0,0,1
4,0,1.0,1,90706,Apartment,1.0,1.0,1.0,80.0,strict,0,0,0,0,1.21,10.0,0,0,1


In [29]:
la_sub['property_type'].value_counts()

Apartment             16434
House                 14389
Condominium            2155
Guesthouse             2013
Townhouse              1379
Guest suite            1249
Bungalow               1198
Loft                    930
Villa                   772
Serviced apartment      391
Hostel                  389
Other                   229
Bed and breakfast       206
Camper/RV               165
Cottage                 151
Boutique hotel          141
Cabin                    87
Tiny house               40
Hotel                    33
Boat                     32
Farm stay                31
Aparthotel               30
Tent                     28
Yurt                     17
Castle                   16
Chalet                   13
Treehouse                12
Earth house               9
Campsite                  7
Tipi                      7
Hut                       6
Dome house                5
Houseboat                 4
Barn                      4
Resort                    3
Bus                 

In [30]:
###binary-encoding for property_type
la_sub['property_type_cleaned'] = la_sub['property_type']
la_sub.loc[(la_sub['property_type'] != 'Apartment') & (la_sub['property_type'] != 'House') 
           & (la_sub['property_type'] != 'Condominium') & (la_sub['property_type'] != 'Guesthouse')
           & (la_sub['property_type'] != 'Townhouse') & (la_sub['property_type'] != 'Guest suite')
           & (la_sub['property_type'] != 'Bungalow') & (la_sub['property_type'] != 'Loft'), 
           'property_type_cleaned'] = 'Other'

In [31]:
la_sub['property_type_cleaned'].value_counts()

Apartment      16434
House          14389
Other           2841
Condominium     2155
Guesthouse      2013
Townhouse       1379
Guest suite     1249
Bungalow        1198
Loft             930
Name: property_type_cleaned, dtype: int64

In [32]:
#one-hot encoding 
binary_encoded1 = pd.get_dummies(la_sub.property_type_cleaned)
newcols1 = binary_encoded1.columns
la_sub[newcols1] = binary_encoded1

In [33]:
# output cleaned listing data 
la_sub.to_csv("listings_cleaned.csv")  # only have a subset of features

In [34]:
la_sub.isna().sum()

host_is_superhost            0
host_total_listings_count    0
host_identity_verified       0
zipcode                      0
property_type                0
bathrooms                    0
bedrooms                     0
beds                         0
price                        0
cancellation_policy          0
availability_30              0
availability_60              0
availability_90              0
availability_365             0
reviews_per_month            0
review_scores_location       0
flexible                     0
moderate                     0
strict                       0
property_type_cleaned        0
Apartment                    0
Bungalow                     0
Condominium                  0
Guest suite                  0
Guesthouse                   0
House                        0
Loft                         0
Other                        0
Townhouse                    0
dtype: int64